In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Data Cleaning and Feature Engineering

In [2]:
# read in csv and adjust header row
# preview random sample of data
df = pd.read_csv('persona_data.csv')
df.columns = df.iloc[0]
df = df[1:]
df.sample(5)

,1. First Name,2. Last Name,3. Age,4. Residence Address,5. Country of ID,6. Device Information,7. IP Address,8. Fraud Status (Ground Truth),9. Verification ID,10. Verification Status,...,17. Compromised Detection,18. MRZ Detection,19. Expired Detection,20. Disallowed Country Detection,21. Entity Detection,22. Portrait Detection,23. Account Comparison,24. Selfie Comparison,25. Extracted Properties Detection,26. Tamper Detection
677,Jeffrey,Jackson,44,"44899 Anderson Flat\nMichelleview, SD 82757",United States,Mozilla/5.0 (iPhone; CPU iPhone OS 3_1_3 like ...,135.114.101.140,FALSE,4511081,passed,...,passed,not_applicable,passed,passed,passed,passed,not_applicable,not_applicable,passed,passed
303,Holly,Hudson,72,"5934 Rollins Gardens\nDonnachester, IL 91525",United States,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,157.62.125.81,FALSE,2139275,failed,...,passed,failed,not_applicable,passed,passed,passed,not_applicable,not_applicable,passed,passed
87,Larry,Mccall,42,"06441 Martinez Stream\nBradville, FL 15904",United States,iPad; CPU iPad OS 9_3_6 like Mac OS X,158.86.166.131,FALSE,6003628,failed,...,passed,not_applicable,passed,passed,passed,passed,not_applicable,not_applicable,passed,passed
748,Jennifer,Norman,63,"5758 Christian Hollow\nWest Cherylton, CT 29769",Japan,iPhone; CPU iPhone OS 7_1_2 like Mac OS X,193.225.89.161,FALSE,4707587,passed,...,passed,not_applicable,passed,passed,passed,passed,not_applicable,not_applicable,passed,passed
716,Mrs.,Christina,68,"37663 Patricia Cove\nDavidland, HI 45128",Jersey,Opera/8.44.(X11; Linux i686; en-ZA) Presto/2.9...,107.69.104.245,Image Quality,3785578,passed,...,passed,passed,passed,passed,passed,passed,not_applicable,not_applicable,passed,passed


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003 entries, 1 to 1003
Data columns (total 26 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   1. First Name                           1003 non-null   object
 1   2. Last Name                            1003 non-null   object
 2   3. Age                                  999 non-null    object
 3   4. Residence Address                    1003 non-null   object
 4   5. Country of ID                        1003 non-null   object
 5   6. Device Information                   1003 non-null   object
 6   7. IP Address                           1003 non-null   object
 7   8. Fraud Status (Ground Truth)          1003 non-null   object
 8   9. Verification ID                      1003 non-null   object
 9   10. Verification Status                 1003 non-null   object
 10  11. Verification Created Date           1003 non-null   object
 11  12. 

In [4]:
# convert Age to int
df['3. Age'] = df['3. Age'].astype('int32', errors='ignore')

In [5]:
# convert Created date to datetime
df['11. Verification Created Date'] = pd.to_datetime(df['11. Verification Created Date'], errors='ignore')

## Extract Geo Info
I think city, state and zip code are the most useful geo info in the 'Residence Address' column. I referenced [this article](https://support.pirateship.com/en/articles/1068456-how-do-i-ship-to-and-from-military-apo-fpo-and-dpo-addresses) to separate out military addresses

### Functions for extracting city, state, and zip 

In [6]:
# get city, state and zipcode from address. 
# Note: Military address will not have city, but will have a zip code.
# --> Military address State Key: AA (Armed Forces Americas)/AE (Armed Forces Europe)/AP (Armed Forces Pacific)
def get_state(row):
    return row.rsplit(' ',2)[1].strip() 

def get_zip(row):
    return row.rsplit(' ',2)[2].strip()

def get_city(row):
    m_city = ['APO','FPO','DPO']
    res = any(ele in row for ele in m_city)
    if res:
        return 'military'
    else:
        address_list = row.rsplit(' ',2)
        city = address_list[0].split('\n')[1].replace(',','').strip()
        return city

In [7]:
# get location of original field to anchor insertion of created fields
address_loc = df.columns.get_loc('4. Residence Address')
print(f'address column is located at index {address_loc}')

address column is located at index 3


In [8]:
# insert state column into dataframe
state = df['4. Residence Address'].apply(get_state)
df.insert(address_loc + 1,'state',state)

In [9]:
# insert zip code column into dataframe
zip_code = df['4. Residence Address'].apply(get_zip)
df.insert(address_loc + 2,'zip_code',zip_code)

In [10]:
# insert city column into dataframe
city = df['4. Residence Address'].apply(get_city)
df.insert(address_loc + 3,'city',city)

## Identify Browser/Device
I used the rules in [this article](https://delib.zendesk.com/hc/en-us/articles/203431259-Browser-device-identification-how-to-find-out-which-browser-and-device-have-been-used-to-submit-a-response) to break out browser and device type from 'Device Information' column


In [11]:
browser_dict = {
    'Firefox':{'must contain':['Firefox'], 
               'must not contain':['Seamonkey']},
    'Seamonkey':{'must contain': ['Seamonkey'], 
                 'must not contain': []},
    'Chrome':{'must contain':['Chrome'], 
              'must not contain': ['Chromium']},
    'Chromium':{'must contain':['Chromium'], 
                'must not contain': []},
    'Safari':{'must contain':['Safari'],
              'must not contain': ['Chrome','Chromium']},
    'Opera':{'must contain':['OPR','Opera'], 
             'must not contain': []},
    'Internet Explorer':{'must contain':['MSIE','Trident','rv:'],
                         'must not contain': []}
}


### Functions for identifying browser and device

In [12]:
def get_browser(row):
    browser = 'Other'
    
    for key in browser_dict:
        
        must_contain = any(ele in row for ele in browser_dict[key].get('must contain'))
        must_not_contain = any(ele in row for ele in browser_dict[key].get('must not contain'))
        if must_contain and not must_not_contain:
            browser = key
            
    return browser

def get_device(row):
    res = any(ele.lower() in row.lower() for ele in mobile_devices)
    if res:
        return 'mobile'
    else:
        return 'desktop'
    
def get_os(row):
    os = 'other'
    for ele in device_os:
        if ele.lower() in row.lower():
            os = ele
    return os

In [13]:
# get location of original field to anchor insertion of created fields
device_loc = df.columns.get_loc('6. Device Information')
print(f'device column is located at index {device_loc}')

device column is located at index 8


In [14]:
# insert browser column into dataframe
browser = df['6. Device Information'].apply(get_browser)
df.insert(device_loc + 1,'browser',browser)

In [15]:
# types of browsers used
df['browser'].unique()

array(['Safari', 'Other', 'Internet Explorer', 'Chrome', 'Opera'],
      dtype=object)

In [16]:
# this should catch most mobile devices 
# Note: if some is using a Android based Smart TV, they may appear in mobile, but this should be rare
# otherwise, lumping all other devices not explititly included in mobile_devices into desktop
mobile_devices = ['iPhone','iPad','iPod','Android','Mobi','Mobile']

In [17]:
# insert device column into dataframe
device = df['6. Device Information'].apply(get_device)
df.insert(device_loc + 2,'device',device)

In [18]:
device_os = ['Mac','Android','Linux','Windows']

In [19]:
os = df['6. Device Information'].apply(get_os)
df.insert(device_loc + 3,'device_os',os)

In [20]:
df['device_os'].unique()

array(['Mac', 'Android', 'Windows', 'Linux'], dtype=object)

## Segment Day Parts

### Function to extract day parts

In [21]:
def get_day_parts(row):
    if row >= 23 or row < 6:
        return 'night'
    if row >= 6 and row < 10:
        return 'early morning'
    if row >= 10 and row < 12:
        return 'late morning'
    if row >= 12 and row < 14:
        return 'early afternoon'
    if row >= 14 and row < 17:
        return 'afternoon'
    if row >= 17 and row < 20:
        return 'evening'
    if row >=20 and row < 23:
        return 'late evening'

In [22]:
# get location of original field to anchor insertion of created fields
created_loc = df.columns.get_loc('11. Verification Created Date')
print(f'created at column is located at index {created_loc}')

created at column is located at index 16


In [23]:
date = df['11. Verification Created Date'].dt.date
df.insert(created_loc + 1,'date',date)

hour = df['11. Verification Created Date'].dt.hour
df.insert(created_loc + 2,'hour',hour)

weekday = df['11. Verification Created Date'].dt.day_name().str[:3]
df.insert(created_loc + 3,'weekday',weekday)

DayOfMonth = df['11. Verification Created Date'].dt.day
df.insert(created_loc + 4,'DayOfMonth',DayOfMonth)

MonthName = df['11. Verification Created Date'].dt.month_name().str[:3]
df.insert(created_loc + 5,'MonthName',MonthName)

Year = df['11. Verification Created Date'].dt.year
df.insert(created_loc + 6,'Year',Year)

day_part = df['hour'].apply(get_day_parts)
df.insert(created_loc + 7,'day_pary',day_part)

## Assess Model
1. I decided to only look at records where the ground truth was observable. 
    - since this represents ~86% of the data, this is acceptable loss
    - perhaps there is value in looking at the predictions on ground truth observations where there were image quality issues. but this seems outside the scope of this project
    

### Functions to build accuracy scores to assess each check as well as overall model

In [24]:
def make_predictor_target_columns(row):
    '''turns predictor and target columns into binary 1/0'''
    
    ones =['TRUE','failed']
    res = any(ele.lower() in row.lower() for ele in ones)
    if res:
        return 1
    else:
        return 0
    
def get_accuracy_score(df,target_col,check_col_index):
    '''
    input: takes dataframe, target col name string, and check col index number
    output: returns check col name, accuracy score for the given check as percent \ 
    and number of observations
    '''
    check_col = df.columns[check_col_index]
    
    # make df to only include evaluated rows
    data = df[(df[check_col] != 'not_applicable') & (pd.notnull(df[check_col]))]
    # make target and predicted series
    target = data[target_col].apply(make_predictor_target_columns)
    predicted = data[check_col].apply(make_predictor_target_columns)
    
    score = np.count_nonzero(target==predicted) / len(target) 
    return check_col, round(score*100,2), data.shape[0]

### Assess overall effectiveness of Persona's automated solution

In [25]:
# look at all records where there are no image quality issues
no_issues = df[df['8. Fraud Status (Ground Truth)'] != 'Image Quality']

In [26]:
# make target and predicted series
target = no_issues['8. Fraud Status (Ground Truth)'].apply(make_predictor_target_columns)
predicted = no_issues['10. Verification Status'].apply(make_predictor_target_columns)

In [27]:
# confusion matrix to assess model outcomes
conf_matrix = pd.DataFrame(confusion_matrix(target, predicted),
                           index = ['actual 0', 'actual 1'], 
                           columns = ['predicted 0', 'predicted 1'])
conf_matrix

,predicted 0,predicted 1
actual 0,474,319
actual 1,9,62


In [28]:
# print classification report 
print(classification_report(target, predicted)) 

              precision    recall  f1-score   support

           0       0.98      0.60      0.74       793
           1       0.16      0.87      0.27        71

    accuracy                           0.62       864
   macro avg       0.57      0.74      0.51       864
weighted avg       0.91      0.62      0.70       864



#### Overall Results
1. Precision is the accuracy of positive predictions --> TP / (TP + FP)
    - basically, of the observations the model thought were of a certain class, what percent were actually of the class
    - 98% of all no-fraud observations were correctly classified
    - however, only 16% of all fraudulent observations we correctly classified
2. Recall is the ability of the classifier to find all positive instances --> TP / (TP + FN)
    - basically, of all the positive observations, what percent did the model identify
    - 60% of the positive no-fraud observations were correctly classified
    - 87% of the positive fraud observations were correctly classified
3. f1-score is the harmonic mean between precision and recall
    - I consider this more relevant for model comparison, so I'll table it for this project
4. support is the number of occurences of a given class in the dataset
    - this is a highly imbalanced data set, so precision and recall are more important than accuracy

### Asses effectiveness of individual checks

In [29]:
# get location of original field
first_check_loc = df.columns.get_loc('12. Color Inconsistency Detection')
print(f'first check column is located at index {first_check_loc}')

first check column is located at index 24


In [30]:
# get location of last field
last_check_loc = df.columns.get_loc(df.columns[-1])
print(f'last check column is located at index {last_check_loc}')

last check column is located at index 38


In [31]:
check_col_names = []
accuracy_scores = []
num_observations = []

for loc in range(first_check_loc,last_check_loc+1):
    try:
        check_col_name, accuracy_score, observations = get_accuracy_score(no_issues, '8. Fraud Status (Ground Truth)', loc)
        check_col_names.append(check_col_name)
        accuracy_scores.append(accuracy_score)
        num_observations.append(observations)
    except:
        print(f'error on {df.columns[loc]}')

error on 24. Selfie Comparison


In [32]:
score_card = pd.DataFrame(list(zip(check_col_names, accuracy_scores, num_observations)), columns=['check', 'score', 'observations'])
score_card

,check,score,observations
0,12. Color Inconsistency Detection,93.17,703
1,13. Barcode Detection,75.52,821
2,14. Extraction Inconsistency Detection,93.30,582
3,15. Inquiry Comparison,85.85,530
4,16. Disallowed Type Detection,81.63,860
5,17. Compromised Detection,91.75,861
6,18. MRZ Detection,53.85,39
7,19. Expired Detection,88.38,809
8,20. Disallowed Country Detection,91.64,861
9,21. Entity Detection,88.62,861


In [33]:
# take only checks with at least 75% of observations
sig_score_card = score_card[score_card['observations'] >= round(len(no_issues)*0.75)]
sig_score_card

,check,score,observations
0,12. Color Inconsistency Detection,93.17,703
1,13. Barcode Detection,75.52,821
4,16. Disallowed Type Detection,81.63,860
5,17. Compromised Detection,91.75,861
7,19. Expired Detection,88.38,809
8,20. Disallowed Country Detection,91.64,861
9,21. Entity Detection,88.62,861
10,22. Portrait Detection,91.07,862
12,25. Extracted Properties Detection,91.75,861
13,26. Tamper Detection,93.74,862


In [34]:
# best predictor
print('the best predictor:')
sig_score_card[sig_score_card.score == sig_score_card.score.max()]

the best predictor:


,check,score,observations
13,26. Tamper Detection,93.74,862


In [35]:
# worst predictor
print('the worst predictor:')
sig_score_card[sig_score_card.score == sig_score_card.score.min()]

the worst predictor:


,check,score,observations
1,13. Barcode Detection,75.52,821


#### Results on all classes
Looking at accuracy over both classes (where the check is valid for at least 75% of the observations), it would appear that...
1. Tamper Detection is the best single check
2. Barcode Detection is the worst single check

HOWEVER...

In [36]:
# look at all records where there are no image quality issues and ONLY fraudulent observations
no_issues_fraud_only = no_issues[no_issues['8. Fraud Status (Ground Truth)'] == 'TRUE']

In [37]:
check_col_names = []
accuracy_scores = []
num_observations = []

for loc in range(first_check_loc,last_check_loc+1):
    try:
        check_col_name, accuracy_score, observations = get_accuracy_score(no_issues_fraud_only, '8. Fraud Status (Ground Truth)', loc)
        check_col_names.append(check_col_name)
        accuracy_scores.append(accuracy_score)
        num_observations.append(observations)
    except:
        print(f'error on {df.columns[loc]}')

error on 24. Selfie Comparison


In [38]:
fraud_only_score_card = pd.DataFrame(list(zip(check_col_names, accuracy_scores, num_observations)), columns=['check', 'score', 'observations'])
fraud_only_score_card

,check,score,observations
0,12. Color Inconsistency Detection,6.25,48
1,13. Barcode Detection,84.06,69
2,14. Extraction Inconsistency Detection,13.33,15
3,15. Inquiry Comparison,27.08,48
4,16. Disallowed Type Detection,18.31,71
5,17. Compromised Detection,1.41,71
6,18. MRZ Detection,100.00,2
7,19. Expired Detection,7.02,57
8,20. Disallowed Country Detection,0.00,71
9,21. Entity Detection,18.31,71


In [39]:
# take only checks with at least 75% of fraudulent observations
sig_fraud_only_score_card = fraud_only_score_card[fraud_only_score_card['observations'] >= round(len(no_issues_fraud_only)*0.75)]
sig_fraud_only_score_card

,check,score,observations
1,13. Barcode Detection,84.06,69
4,16. Disallowed Type Detection,18.31,71
5,17. Compromised Detection,1.41,71
7,19. Expired Detection,7.02,57
8,20. Disallowed Country Detection,0.00,71
9,21. Entity Detection,18.31,71
10,22. Portrait Detection,8.45,71
12,25. Extracted Properties Detection,0.00,71
13,26. Tamper Detection,23.94,71


In [40]:
# best predictor
print('the best predictor:')
sig_fraud_only_score_card[sig_fraud_only_score_card.score == sig_fraud_only_score_card.score.max()]

the best predictor:


,check,score,observations
1,13. Barcode Detection,84.06,69


In [41]:
# worst predictor
print('the worst predictor:')
sig_fraud_only_score_card[sig_fraud_only_score_card.score == sig_fraud_only_score_card.score.min()]

the worst predictor:


,check,score,observations
8,20. Disallowed Country Detection,0.0,71
12,25. Extracted Properties Detection,0.0,71


#### Results on fraudulent observations only
Looking at accuracy over fraud class (where the check is valid for at least 75% of the observations), it reveals that...
1. Barcode Detection is actually the best single check!
2. Disallowed Country and Extracted Properties Detection tie for the worst performers


In [42]:
# df.to_csv('persona_data_clean.csv',index=False)